This notebook we will use to find and deploy the most affordable GPU's on the market with the Shadeform API. We limit our search to machines with 1x80GB A100 machines, but that is easily configurable below.

This notebook leverages the docker container created under `llm-eval-harness-benchmarking` to run basic huggingface benchmarks for a given huggingface model.

It also re-uses code from [find_and_use_gpus.ipynb](link.com) for using the Shadeform API to find available instances.

In [1]:
import requests
import json


/Users/alexowen/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
base_url = "https://api.shadeform.ai/v1/instances"
instance_type_url = base_url + "/types"
create_url = base_url + "/create"
headers = {
    "X-API-KEY": "wMKxY8d52U3EHmFYnd8qr5y6", 
    "Content-Type" : "application/json"
}
shade_instance_type = "A6000"
gpu_type = "A6000"
num_gpus = 1

params = {
    'gpu_type' : gpu_type,
    'sort' : 'price',
    'available' : True,
    'num_gpus' : num_gpus
}



In [3]:
response = requests.request("GET", instance_type_url, headers=headers, params=params)
instance_types = json.loads(response.text)["instance_types"]
best_instance = None
region = None
if len(instance_types) > 0:
    best_instance = instance_types[0]
    region = best_instance['availability'][0]['region']
    print(f"The cheapest {gpu_type} instance with {num_gpus} gpu(s) is:", best_instance)
else:
    print(f"No instances of type {gpu_type} instance with {num_gpus} gpu(s) found.")


The cheapest A6000 instance with 1 gpu(s) is: {'cloud': 'massedcompute', 'shade_instance_type': 'A6000', 'cloud_instance_type': 'gpu_1x_a6000', 'configuration': {'memory_in_gb': 48, 'storage_in_gb': 256, 'vcpus': 6, 'num_gpus': 1, 'gpu_type': 'A6000', 'interconnect': 'pcie', 'os_options': ['ubuntu22.04_cuda12.2_shade_os'], 'vram_per_gpu_in_gb': 48}, 'memory_in_gb': 48, 'storage_in_gb': 256, 'vcpus': 6, 'num_gpus': 1, 'gpu_type': 'A6000', 'interconnect': 'pcie', 'hourly_price': 57, 'availability': [{'region': 'us-central-2', 'available': True}]}


In [4]:
payload = {
  "cloud": best_instance["cloud"],
  "region": region,
  "shade_instance_type": shade_instance_type,
  "shade_cloud": True,
  "name": "cool_gpu_server",
  "launch_configuration": {
    "type": "docker",
    "docker_configuration": {
      "image": "shadeform/lm-eval-harness",
      "envs": [
      	{
	      	"name": "model",
	      	"value": "mistralai/Mistral-7B-Instruct-v0.2"
      	},
      	{
	      	"name": "tasks",
	      	"value": "hellaswag"
      	},
      ]
    }
  }
}

In [5]:
#request the best instance that is available
response = requests.request("POST", create_url, json=payload, headers=headers)


In [6]:
#easy way to visually see if this request worked
print(response.text)

{"id":"4e90f11a-e098-4b11-a9fa-ebc381e9bc9a","cloud_assigned_id":"c23ad075-9615-49b9-9a89-e8ee743a12b2"}


In [8]:
instance_response = requests.request("GET", base_url, headers=headers)

print(instance_response.text)
instance = json.loads(instance_response.text)["instances"][0]
instance_status = instance['status']
if instance_status == 'active':
    print(f"Instance is active with IP: {instance['ip']}")
else:
    print(f"Instance isn't yet active: {instance}" )

{"instances":[{"id":"4e90f11a-e098-4b11-a9fa-ebc381e9bc9a","cloud":"massedcompute","region":"us-central-2","shade_instance_type":"A6000","cloud_instance_type":"gpu_1x_a6000","cloud_assigned_id":"c23ad075-9615-49b9-9a89-e8ee743a12b2","shade_cloud":true,"name":"cool_gpu_server","configuration":{"memory_in_gb":48,"storage_in_gb":256,"vcpus":6,"num_gpus":1,"gpu_type":"A6000","interconnect":"pcie","os":"ubuntu22.04_cuda12.2_shade_os","vram_per_gpu_in_gb":48},"ip":"64.247.196.46","ssh_user":"shadeform","ssh_port":22,"status":"active","status_details":"Container run started... View docker progress logs by running 'journalctl -u init-script'","cost_estimate":"0.0335054514","hourly_price":57,"created_at":"2024-02-21T18:03:42.911404Z","deleted_at":null,"launch_configuration":{"type":"docker","docker_configuration":{"image":"shadeform/lm-eval-harness","envs":[{"name":"tasks","value":"hellaswag"},{"name":"model","value":"mistralai/Mistral-7B-Instruct-v0.2"}]}}}]}
Instance is active with IP: 64.247